RNN sentiment analysis

In [ ]:
import numpy as np
import pandas as pd
import codecs
import keras
import h5py
import jax
import jax.numpy as jnp #gonna try use jax since its supposedly faster
import glob
import codecs
import string
import tensorflow as tf
from tensorflow import keras

# Keras model, layer and preprocessing libraries
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import load_model
from keras.preprocessing import sequence

# NLTK imports
from nltk.tokenize import RegexpTokenizer

# Keras model, layer and preprocessing libraries
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import load_model
from keras.preprocessing import sequence

# NLTK imports
from nltk.tokenize import RegexpTokenizer

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os

cwd = os.getcwd()  

# loadin DATA


In [ ]:
files = os.listdir(cwd+"/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec") 
files

In [ ]:
os.chdir("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec")


extension = 'csv'
all_vectors = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_vectors = pd.concat([pd.read_csv(f) for f in all_vectors])

In [ ]:
len(combined_vectors)

163976

# Training the model

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#!unzip -u "/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/glove.6B.zip" -d "/content/gdrive/MyDrive/CS4225/ML team/pre-processed data"

Combining dictionary and sentiment labels into one dataframe and write to CSV

In [ ]:
def read_data(path):
  # read dictionary into df
  df_data_sentence = pd.read_table(path + "dictionary.txt")
  df_data_sentence_processed = df_data_sentence["Phrase|Index"].str.split("|", expand=True)
  df_data_sentence_processed = df_data_sentence_processed.rename(columns={0: "Phrase", 1: "phrase_ids"})
  # read sentiment labels into df
  df_data_sentiment = pd.read_table(path + "sentiment_labels.txt")
  df_data_sentiment_processed = df_data_sentiment["phrase ids|sentiment values"].str.split("|", expand=True)
  df_data_sentiment_processed = df_data_sentiment_processed.rename(columns={0: "phrase_ids", 1: "sentiment_values"})
  #combine data frames containing sentence and sentiment
  df_processed_all = df_data_sentence_processed.merge(df_data_sentiment_processed, how="inner", on="phrase_ids")

  return df_processed_all



In [ ]:
# Path to the data
dictFile = '/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/dictionary.txt'
sentimentFile = '/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/sentiment_labels.txt'

# Read dictionary into pandas dataframe
df_dictionary = pd.read_table(dictFile)
df_dictionary_processed = df_dictionary['!|0'].str.split('|', expand = True)  #damn code... is supposed to be ! / 0
df_dictionary_processed = df_dictionary_processed.rename(columns = {0: 'Phrase', 1: 'phrase_ids'})

# Read sentiment labels into pandas dataframe
df_sentiments = pd.read_table(sentimentFile)
df_sentiments_processed = df_sentiments['phrase ids|sentiment values'].str.split('|', expand = True)
df_sentiments_processed = df_sentiments_processed.rename(columns = {0: 'phrase_ids', 1: 'sentiment_values'})

# Combine the dictionary and sentiment data into one dataframe
df_all = df_dictionary_processed.merge(df_sentiments_processed, on = 'phrase_ids', how = 'inner')

df_all.head()

,Phrase,phrase_ids,sentiment_values
0,! ',22935,0.52778
1,! '',18235,0.5
2,! Alas,179257,0.44444
3,! Brilliant,22936,0.86111
4,! Brilliant !,40532,0.93056


In [ ]:
phrase = df_all['Phrase']
sentiment_values = df_all['sentiment_values']
phrase.head()

0              ! '
1             ! ''
2           ! Alas
3      ! Brilliant
4    ! Brilliant !
Name: Phrase, dtype: object

In [ ]:
phrase_lower = phrase.apply(lambda x: x.lower())
# phrase_punctuation = phrase_lower.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
# phrase_remove_spec_char = phrase_punctuation.apply(lambda x: re.sub("[^a-zA-Z0-9., ]+", "", x))
phrase_lower.head()

0              ! '
1             ! ''
2           ! alas
3      ! brilliant
4    ! brilliant !
Name: Phrase, dtype: object

In [ ]:
df_all = pd.concat((phrase_lower, sentiment_values), axis = 1)
df_all_processed = df_all[df_all['Phrase'].map(len) > 1]
df_all_processed.head()

,Phrase,sentiment_values
0,! ',0.52778
1,! '',0.5
2,! alas,0.44444
3,! brilliant,0.86111
4,! brilliant !,0.93056


In [ ]:
df_all_processed.to_csv("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/processed_dictionary_sentiments.csv")

Load processed dictionary sentiments csv

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/processed_dictionary_sentiments.csv")

# Split the data to get 50% for training using uniform distribution
train_indices = np.random.rand(len(df)) < 0.5
train_data = df[train_indices]
test_and_val_data = df[~train_indices]

# Split the leftover dataset into 25% validation and 25% testing using uniform distribution
test_indices = np.random.rand(len(test_and_val_data)) < 0.5
test_data = test_and_val_data[test_indices]
val_data = test_and_val_data[~test_indices]

train_data.to_csv("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/Training.csv")
test_data.to_csv("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/Test.csv")
val_data.to_csv("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/Validation.csv")

load in dctionary

In [ ]:
"""path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))"""

'path_to_glove_file = os.path.join(\n    os.path.expanduser("~"), "/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/glove.6B.100d.txt"\n)\n\nembeddings_index = {}\nwith open(path_to_glove_file) as f:\n    for line in f:\n        word, coefs = line.split(maxsplit=1)\n        coefs = np.fromstring(coefs, "f", sep=" ")\n        embeddings_index[word] = coefs\n\nprint("Found %s word vectors." % len(embeddings_index))'

In [ ]:
# Path to the data
gloveFile = "/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/glove.6B.100d.txt"

word_embedding_dict = {}
word_index = {}

with codecs.open(gloveFile, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        # The word that is to be the key
        word = values[0]
        # Update word_index
        word_index[word] = len(word_embedding_dict)
        # The vector that represents the word
        vector = np.asarray(values[1:], "float32")
        word_embedding_dict[word] = vector

Create weight matrix

In [ ]:
word_embedding_matrix = np.zeros((len(word_embedding_dict), 100))
word_embedding_matrix.shape

(400000, 100)

In [ ]:
i = 0
for word in word_embedding_dict.keys():
    if i > len(word_embedding_matrix):
        break
    
    word_embedding_vector = word_embedding_dict[word]
    if word_embedding_vector is not None:
        word_embedding_matrix[i] = word_embedding_vector
        i = i + 1

Load data matrix

In [ ]:
def data_matrix(data): #28 mins
    data_processed = np.zeros((len(data), 280), dtype = 'int32')
    index = 0
    
    # Convert keys to lowercase
    word_index_lower = {k.lower(): v for k, v in word_index.items()}
    
    # Extracting out the phrases
    sentences = data['Phrase']
    
    # Placing the sentences into the data_matrix
    for sentence in sentences:
        tokenizer = RegexpTokenizer(r'\w+')
        sentence_words = tokenizer.tokenize(sentence)
        
        # Capture all the words into data_matrix
        i = 0
        for word in sentence_words:
            word_lower = word.lower()
            try:
                data_processed[index][i] = word_index_lower[word_lower]
            except Exception as e:
                if str(e) == word:
                    data_processed[index][i] = 0
                continue
            i = i + 1
        index = index + 1
    
    print("Completed")
    return data_processed
    
train_data_x = data_matrix(train_data)
test_data_x = data_matrix(test_data)
val_data_x = data_matrix(val_data)

Load labels matrix

In [ ]:
def labels_matrix(data): 
    data_here = data.copy()
    labels = data_here['sentiment_values']

    # Convert to float
    labels_float = labels.astype(float)
    
    # Categories of 0, 1 and 2 with 0 being negative, 1 being neutral and 2 being positive
    categories = ['0', '1', '2']
    
    # Categorise the labels 
    labels_categories = [-1 for i in range(len(labels_float))]
    i = 0
    for label in labels_float:
        if label < 0.5:
            labels_categories[i] = 0
        if label == 0.5:
            labels_categories[i] = 1
        if label > 0.5:
            labels_categories[i] = 2
        i = i + 1
    
    # Placing labels_category as part of the dataframe
    data_here['category'] = labels_categories
    
    # Getting matrix out of the category column
    labels_mult = data_here['category'].astype(int)
    dummies = pd.get_dummies(labels_mult, prefix = '', prefix_sep = '')
    dummies = dummies.T.reindex(categories).T.fillna(0)
    labels_matrix = dummies.to_numpy()
    
    print("Completed")
    
    return labels_matrix

train_data_y = labels_matrix(train_data)
test_data_y = labels_matrix(test_data)
val_data_y = labels_matrix(val_data)

Completed
Completed
Completed


Develop LSTM Model

In [ ]:
# Developing the LSTM Model 
model = Sequential()
# Input_length of 280 is chosen as 280 characters are allowed in twitter
model.add(Embedding(len(word_embedding_matrix), 100, weights = [word_embedding_matrix], input_length = 280, trainable = False))
model.add(Bidirectional(LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.50))
model.add(Dense(3, activation = 'softmax'))

# Adam's optimiser
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Training the LSTM Model

In [ ]:
# save the best model and early stopping
saveBestModel = keras.callbacks.ModelCheckpoint('/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/RNN.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [ ]:
# Fit the model
model.fit(train_data_x, train_data_y, batch_size=1000, epochs=10,validation_data=(val_data_x, val_data_y), callbacks=[saveBestModel, earlyStopping])

In [ ]:
# Model Evaluation
score, accuracy = model.evaluate(test_data_x, test_data_y, batch_size = 500)

print('Test score:', score)
print('Test accuracy:', accuracy)

120/120 [==============================] - 192s 2s/step - loss: 0.6939 - accuracy: 0.6955
Test score: 0.6939077377319336
Test accuracy: 0.6955015063285828


Saving the LSTM Model

In [ ]:
model.save_weights("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/RNN.h5")
print("Saved model to disk")

# Prediction model

In [ ]:
# Path to the data
gloveFile = '/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/glove.6B.100d.txt'

word_embedding_dict = {}
word_index = {}

with codecs.open(gloveFile, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        # The word that is to be the key
        word = values[0]
        # Update word_index
        word_index[word] = len(word_embedding_dict)
        # The vector that represents the word
        vector = np.asarray(values[1:], "float32")
        word_embedding_dict[word] = vector

In [ ]:
word_embedding_matrix = np.zeros((len(word_embedding_dict), 100))

i = 0
for word in word_embedding_dict.keys():
    if i > len(word_embedding_matrix):
        break
    
    word_embedding_vector = word_embedding_dict[word]
    if word_embedding_vector is not None:
        word_embedding_matrix[i] = word_embedding_vector
        i = i + 1

In [ ]:
# Developing the LSTM Model 
model = Sequential()
# Input_length of 280 is chosen as 280 characters are allowed in twitter
model.add(Embedding(len(word_embedding_matrix), 100, weights = [word_embedding_matrix], input_length = 280, trainable = False))
model.add(Bidirectional(LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.50))
model.add(Dense(3, activation = 'softmax'))

# Adam's optimiser
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 280, 100)          40000000  
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense (Dense)               (None, 512)               131584    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 40,367,619
Trainable params: 367,619
Non-trainable params: 40,000,000
____________________________________

Sort by date

In [ ]:
def get_sentiment_date(data, word_index):
    # Load the RNN model
    model.load_weights("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/RNN.h5")
    print("Model weight loaded successfully")
    
    # Convert keys to lowercase
    word_index_lower = {k.lower(): v for k, v in word_index.items()}
    
    # Preparing the data for prediction
    word_list = np.zeros((1, 280), dtype = 'int32')
    sentiment_analysis = {}
    
    for index, indiv_data in data.iterrows():

        sentence = indiv_data['text']
        print(indiv_data['text'])
        tokenizer = RegexpTokenizer(r'\w+')
        sentence_words = tokenizer.tokenize(str(sentence))
        
        # Capture all the word into one list for prediction
        i = 0
        for word in sentence_words:
            word_lower = word.lower()
            try:
                word_list[0][i] = word_index_lower[word_lower]
            except Exception as e:
                if str(e) == word:
                    word_list[0][i] = 0
                continue
            i = i + 1
        
        # Predict the score of the tweet 
        score = model.predict(word_list, batch_size = len(data), verbose = 0)
        max_score = np.argmax(score)

        # Check the date of the tweet
        date = indiv_data['created_at'][0:10]
        
        # Set all the dictionary of the date to be 0 if it is a new found date
        if sentiment_analysis.get(date) is None:
            sentiment_analysis[date] = {}
            sentiment_analysis[date]['negative'] = 0
            sentiment_analysis[date]['neutral'] = 0
            sentiment_analysis[date]['positive'] = 0
            
        # Incrementing the negative, neutral or positive tweets respectively
        if max_score == 0:
            sentiment_analysis[date]['negative'] += 1
        if max_score == 1:
            sentiment_analysis[date]['neutral'] += 1
        if max_score == 2: 
            sentiment_analysis[date]['positive'] += 1 
            
    return sentiment_analysis

# Testing the function created
# data_file = pd.read_csv("Test/hello.csv")
# sentiment_analysis = get_sentiment(data_file, word_index)
# print(sentiment_analysis)

sort by sentence

In [ ]:
def get_sentiment_sentence(data, word_index):
    # Load the RNN model
    model.load_weights("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/RNN_Split_Dataset/RNN.h5")
    print("Model weight loaded successfully")
    
    # Convert keys to lowercase
    word_index_lower = {k.lower(): v for k, v in word_index.items()}
    
    # Preparing the data for prediction
    word_list = np.zeros((1, 280), dtype = 'int32')
    sentiment_analysis = {}
    
    for index, indiv_data in data.iterrows():

        sentence = indiv_data['text']
        #print(indiv_data['text'])
        tokenizer = RegexpTokenizer(r'\w+')
        sentence_words = tokenizer.tokenize(str(sentence))
        
        
        # Capture all the word into one list for prediction
        i = 0
        for word in sentence_words:
            word_lower = word.lower()
            try:
                word_list[0][i] = word_index_lower[word_lower]
            except Exception as e:
                if str(e) == word:
                    word_list[0][i] = 0
                continue
            i = i + 1
        
        # Predict the score of the tweet 
        score = model.predict(word_list, batch_size = len(data), verbose = 0)
        single_score = np.round(np.argmax(score)/10, decimals=2) # maximum of the array i.e single band
        # weighted score of top 3 bands
        top_3_index = np.argsort(score)[0][-3:]
        top_3_scores = score[0][top_3_index]
        top_3_weights = top_3_scores/np.sum(top_3_scores)
        single_score_dot = np.round(np.dot(top_3_index, top_3_weights)/10, decimals = 2)
        #print(single_score_dot, single_score)
        #sentiment_analysis = sentiment_analysis.append({'Unnamed: 0': index,'Score': single_score_dot}, ignore_index=True)

        sentiment_analysis[str(index)] = single_score_dot


            
    return sentiment_analysis

In [ ]:
tester = pd.read_csv("/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/ae.csv")
tester_1 = tester

In [ ]:
import time

this is only for one file, tester

In [ ]:
t1 = time.perf_counter()

### Your code goes here ###
results = get_sentiment_sentence(tester_1, word_index)

t2 = time.perf_counter()
print('time taken to run:',t2-t1)

Model weight loaded successfully
time taken to run: 26.81105659399998


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
data_sample = tester['text'][5]
sid = SentimentIntensityAnalyzer()
ss = sid.polarity_scores(data_sample)
ss['compound']

0.0

In [ ]:
# Path to the processed tweets 
csv_path = cwd+"/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/*.csv"

# List of outcomes
outcomes = {}

# Loop through all csvs in the folder
for file in glob.glob(csv_path):
    print("Currently at: " + file)
    t1 = time.perf_counter()

    # Get the sentiments for the csv file
    data = pd.read_csv(file)
    results = get_sentiment_sentence(data, word_index)

    file_name = str(file)[58:-4]
    outcomes[str(file_name)] = results

    t2 = time.perf_counter()
    print('time taken to run:',t2-t1)
    
    # Obtain the values and keys from the dictionary returned from get_sentiment function


Currently at: /content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/ae.csv
Model weight loaded successfully
time taken to run: 28.87263968000002
Currently at: /content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/af.csv
Model weight loaded successfully
time taken to run: 23.647138899000083
Currently at: /content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/ag.csv
Model weight loaded successfully
time taken to run: 38.45871867699998
Currently at: /content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/ar.csv
Model weight loaded successfully
time taken to run: 9.947539934999895
Currently at: /content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/ax.csv
Model weight loaded successfully
time taken to run: 43.272508431999995
Currently at: /content/gdrive/MyDrive/CS4225/ML team/pre-processed data/twitter/scrap_nov_dec/az.csv
Model weight loaded successfully
time take

In [ ]:
path = '/content/gdrive/MyDrive/CS4225/ML team/pre-processed data/stanfordSentimentTreebank/output_nov_dec.csv'

data_items = outcomes.items()
data_list = list(data_items)

final_df = pd.DataFrame(data_list)

with open(path, 'w', encoding = 'utf-8-sig') as f:
  final_df.to_csv(f)